## 1. Configuração do DuckDB

In [12]:
import duckdb
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [13]:
conn = duckdb.connect(':memory:')

## 2. Carregamento dos Dados Tratados

In [14]:
# Carregando os dados tratados
consumo_df = pd.read_csv('../data/consumo_tratado.csv')
clima_df = pd.read_csv('../data/clima_tratado.csv')
clientes_df = pd.read_csv('../data/clientes_tratado.csv')


consumo_df['date'] = pd.to_datetime(consumo_df['date'])
clima_df['date'] = pd.to_datetime(clima_df['date'])

## 3. DuckDB

In [15]:
# Criando tabela de clientes
conn.execute("DROP TABLE IF EXISTS consumo")
conn.execute("DROP TABLE IF EXISTS clima")  
conn.execute("DROP TABLE IF EXISTS clientes")

# Tabela de Clientes
conn.execute("""
    CREATE TABLE clientes (
        client_id VARCHAR PRIMARY KEY,
        region VARCHAR NOT NULL
    );
""")

# Inserindo dados de clientes
conn.execute("INSERT INTO clientes SELECT * FROM clientes_df")

# Tabela de Clima
conn.execute("""
    CREATE TABLE clima (
        region VARCHAR NOT NULL,
        date DATE NOT NULL,
        temperature DOUBLE NOT NULL,
        humidity DOUBLE NOT NULL,
        PRIMARY KEY (region, date)
    );
""")

# Inserindo dados de clima
conn.execute("INSERT INTO clima SELECT * FROM clima_df")

# Tabela de Consumo
conn.execute("""
    CREATE TABLE consumo (
        client_id VARCHAR NOT NULL,
        date DATE NOT NULL,
        consumption_kwh DOUBLE NOT NULL,
        PRIMARY KEY (client_id, date),
        FOREIGN KEY (client_id) REFERENCES clientes(client_id)
    );
""")

# Inserindo dados de consumo
conn.execute("INSERT INTO consumo SELECT * FROM consumo_df")

## 4. Validação das Tabelas

In [16]:
# Validando as tabelas criadas

# Listando tabelas
tabelas = conn.execute("SHOW TABLES").fetchall()
print(f"{[t[0] for t in tabelas]}")

# Contagem de registros
for tabela in ['clientes', 'clima', 'consumo']:
    count = conn.execute(f"SELECT COUNT(*) FROM {tabela}").fetchone()[0]
    print(f"{tabela}: {count:,} registros")

# Verificando integridade
clientes_sem_consumo = conn.execute("""
    SELECT COUNT(*) FROM clientes c 
    WHERE NOT EXISTS (SELECT 1 FROM consumo co WHERE co.client_id = c.client_id)
""").fetchone()[0]
print(f"Clientes sem dados de consumo: {clientes_sem_consumo}")

clientes_sem_clima = conn.execute("""
    SELECT COUNT(*) FROM clientes c 
    WHERE NOT EXISTS (SELECT 1 FROM clima cl WHERE cl.region = c.region)
""").fetchone()[0]
print(f"Clientes sem dados climáticos: {clientes_sem_clima}")

['clientes', 'clima', 'consumo']
clientes: 100 registros
clima: 900 registros
consumo: 18,000 registros
Clientes sem dados de consumo: 0
Clientes sem dados climáticos: 0


## 5. Insights

In [17]:
# Consultas para geração de insights

#Consumo médio por região
resultado = conn.execute("""
    SELECT 
        cl.region,
        COUNT(DISTINCT c.client_id) as num_clientes,
        ROUND(AVG(co.consumption_kwh), 2) as consumo_medio,
        ROUND(STDDEV(co.consumption_kwh), 2) as desvio_padrao,
        ROUND(MIN(co.consumption_kwh), 2) as consumo_min,
        ROUND(MAX(co.consumption_kwh), 2) as consumo_max
    FROM consumo co
    JOIN clientes c ON co.client_id = c.client_id
    JOIN clima cl ON c.region = cl.region AND co.date = cl.date
    GROUP BY cl.region
    ORDER BY consumo_medio DESC
""").fetchdf()
print(resultado.to_string(index=False))

region  num_clientes  consumo_medio  desvio_padrao  consumo_min  consumo_max
 Oeste            17          15.96           3.44         3.81        27.35
 Leste            18          14.97           4.11         3.29        27.92
Centro            17          14.96           3.28         3.65        25.17
   Sul            15          14.69           3.68         4.29        26.14
 Norte            33          14.11           3.73         2.64        25.32


In [18]:
# Correlação temporal - consumo vs temperatura
resultado = conn.execute("""
    SELECT 
        EXTRACT(MONTH FROM co.date) as mes,
        ROUND(AVG(co.consumption_kwh), 2) as consumo_medio,
        ROUND(AVG(cl.temperature), 2) as temperatura_media,
        ROUND(AVG(cl.humidity), 2) as umidade_media,
        COUNT(*) as registros
    FROM consumo co
    JOIN clientes c ON co.client_id = c.client_id
    JOIN clima cl ON c.region = cl.region AND co.date = cl.date
    GROUP BY EXTRACT(MONTH FROM co.date)
    ORDER BY mes
""").fetchdf()
print(resultado.to_string(index=False))

 mes  consumo_medio  temperatura_media  umidade_media  registros
   1          14.82              25.07          59.79       3100
   2          14.75              25.13          60.30       2800
   3          14.81              25.18          60.08       3100
   4          14.84              25.10          60.06       3000
   5          14.77              25.02          59.94       3100
   6          14.87              24.99          60.14       2900


In [19]:
# Top 10 dias com maior consumo
resultado = conn.execute("""
    SELECT 
        co.date,
        ROUND(AVG(co.consumption_kwh), 2) as consumo_medio_dia,
        ROUND(AVG(cl.temperature), 2) as temperatura_media,
        ROUND(AVG(cl.humidity), 2) as umidade_media,
        EXTRACT(DOW FROM co.date) as dia_semana
    FROM consumo co
    JOIN clientes c ON co.client_id = c.client_id
    JOIN clima cl ON c.region = cl.region AND co.date = cl.date
    GROUP BY co.date
    ORDER BY consumo_medio_dia DESC
    LIMIT 10
""").fetchdf()
print(resultado.to_string(index=False))

      date  consumo_medio_dia  temperatura_media  umidade_media  dia_semana
2023-01-03              15.49              23.69          60.83           2
2023-04-07              15.37              24.46          60.04           5
2023-06-05              15.35              24.80          59.08           1
2023-01-05              15.33              24.75          60.32           4
2023-06-08              15.26              24.18          59.46           4
2023-05-15              15.26              23.97          60.32           1
2023-03-21              15.23              25.20          59.14           2
2023-01-08              15.22              25.04          61.57           0
2023-06-27              15.21              25.29          61.07           2
2023-02-19              15.20              24.75          61.56           0


## 6. Feature Engineering

In [20]:
# Criando view consolidada com features para modelagem
# View principal com todas as features e a criação da sensação térmica
conn.execute("""
    DROP VIEW IF EXISTS dataset_modelagem;
    CREATE VIEW dataset_modelagem AS
    SELECT 
        co.client_id,
        co.date,
        co.consumption_kwh as target,
        cl.temperature,
        cl.humidity,
        -- Calculando sensação térmica (Heat Index) diretamente no SQL
        CASE 
            WHEN ((cl.temperature * 9/5) + 32) < 80 THEN cl.temperature
            ELSE ROUND(
                ((-42.379 + 
                  2.04901523 * ((cl.temperature * 9/5) + 32) + 
                  10.14333127 * cl.humidity - 
                  0.22475541 * ((cl.temperature * 9/5) + 32) * cl.humidity - 
                  6.83783e-3 * POWER(((cl.temperature * 9/5) + 32), 2) - 
                  5.481717e-2 * POWER(cl.humidity, 2) + 
                  1.22874e-3 * POWER(((cl.temperature * 9/5) + 32), 2) * cl.humidity + 
                  8.5282e-4 * ((cl.temperature * 9/5) + 32) * POWER(cl.humidity, 2) - 
                  1.99e-6 * POWER(((cl.temperature * 9/5) + 32), 2) * POWER(cl.humidity, 2)
                ) - 32) * 5/9, 2)
        END as sensacao_termica,
        c.region,
        EXTRACT(YEAR FROM co.date) as ano,
        EXTRACT(MONTH FROM co.date) as mes,
        EXTRACT(DAY FROM co.date) as dia,
        EXTRACT(DOW FROM co.date) as dia_da_semana,
        EXTRACT(DOY FROM co.date) as dia_do_ano,
        LAG(co.consumption_kwh, 1) OVER (PARTITION BY co.client_id ORDER BY co.date) as consumption_lag1,
        LAG(co.consumption_kwh, 7) OVER (PARTITION BY co.client_id ORDER BY co.date) as consumption_lag7,
        AVG(co.consumption_kwh) OVER (PARTITION BY co.client_id ORDER BY co.date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as consumption_ma7
    FROM consumo co
    JOIN clientes c ON co.client_id = c.client_id
    JOIN clima cl ON c.region = cl.region AND co.date = cl.date
    ORDER BY co.client_id, co.date
""")

# Verificando a view
resultado = conn.execute("SELECT COUNT(*) as total_records FROM dataset_modelagem").fetchone()

# Mostrando amostra das features
amostra = conn.execute("""
    SELECT 
        client_id, date, target, temperature, humidity, sensacao_termica, region,
        mes, dia, dia_da_semana, dia_do_ano, consumption_lag1, consumption_ma7
    FROM dataset_modelagem 
    WHERE consumption_lag1 IS NOT NULL
    LIMIT 5
""").fetchdf()
print(amostra.to_string(index=False))

client_id       date  target  temperature  humidity  sensacao_termica region  mes  dia  dia_da_semana  dia_do_ano  consumption_lag1  consumption_ma7
    C0000 2023-01-02   16.63    27.090625 59.206250         28.140000  Norte    1    2              1           2             18.64        17.635000
    C0000 2023-01-03   18.11    24.325000 59.365625         24.325000  Norte    1    3              2           3             16.63        17.793333
    C0000 2023-01-04   18.25    23.987500 57.340625         23.987500  Norte    1    4              3           4             18.11        17.907500
    C0000 2023-01-05   19.81    25.200000 59.921875         25.200000  Norte    1    5              4           5             18.25        18.288000
    C0000 2023-01-06   15.87    24.896875 60.596875         24.896875  Norte    1    6              5           6             19.81        17.885000


In [21]:
# Análise de correlações e estatísticas
# Correlação entre consumo e variáveis climáticas por região
correlacao = conn.execute("""
    SELECT 
        region,
        ROUND(CORR(target, temperature), 4) as corr_temp,
        ROUND(CORR(target, humidity), 4) as corr_humidity,
        ROUND(CORR(target, sensacao_termica), 4) as corr_sensacao,
        COUNT(*) as registros
    FROM dataset_modelagem
    WHERE consumption_lag1 IS NOT NULL
    GROUP BY region
    ORDER BY corr_sensacao DESC
""").fetchdf()
print(correlacao.to_string(index=False))

# Correlação geral
corr_geral = conn.execute("""
    SELECT 
        ROUND(CORR(target, temperature), 4) as corr_temperatura,
        ROUND(CORR(target, humidity), 4) as corr_umidade,
        ROUND(CORR(target, sensacao_termica), 4) as corr_sensacao_termica
    FROM dataset_modelagem
    WHERE consumption_lag1 IS NOT NULL
""").fetchdf()
print(corr_geral.to_string(index=False))

# Estatísticas das features criadas
estatisticas = conn.execute("""
    SELECT 
        COUNT(*) as total_registros,
        COUNT(consumption_lag1) as registros_com_lag,
        ROUND(AVG(target), 2) as consumo_medio,
        ROUND(STDDEV(target), 2) as consumo_desvio,
        ROUND(AVG(temperature), 2) as temp_media,
        ROUND(AVG(humidity), 2) as umidade_media,
        ROUND(AVG(sensacao_termica), 2) as sensacao_media,
        ROUND(STDDEV(sensacao_termica), 2) as sensacao_desvio
    FROM dataset_modelagem
""").fetchdf()
print(estatisticas.to_string(index=False))

# Exportando dataset final para CSV
dataset_final = conn.execute("""
    SELECT * FROM dataset_modelagem 
    WHERE consumption_lag1 IS NOT NULL
""").fetchdf()

dataset_final.to_csv('../data/dataset_modelagem.csv', index=False)

region  corr_temp  corr_humidity  corr_sensacao  registros
 Norte    -0.0387         0.0075        -0.0385       5907
 Leste    -0.0571         0.0072        -0.0577       3222
Centro    -0.0614         0.0076        -0.0599       3043
 Oeste    -0.0640        -0.0089        -0.0648       3043
   Sul    -0.0762        -0.0088        -0.0749       2685
 corr_temperatura  corr_umidade  corr_sensacao_termica
          -0.0452       -0.0008                -0.0427
 total_registros  registros_com_lag  consumo_medio  consumo_desvio  temp_media  umidade_media  sensacao_media  sensacao_desvio
           18000              17900          14.81            3.73       25.08          60.05           25.19             1.35


In [22]:
# Fechando conexão
conn.close()